# Famille type

Objectif : Première prise en main d'Open fisca France. 

- Importer OPenfisca
- Créer un cas type de base
- Calculer revenu disponible, irpp, etc
- Simuler une réforme


## Import d'openfisca

### Import des packages


In [2]:
from openfisca_france import FranceTaxBenefitSystem

### Import du système sociofiscal français

In [4]:
tax_benefit_system = FranceTaxBenefitSystem()

In [57]:
scenario = tax_benefit_system.new_scenario()

In [95]:
tax_benefit_system.parameters.impot_revenu.bareme.brackets[1]

threshold:
  2015-01-01: 9700.0
  2014-01-01: 9690.0
  2013-01-01: 6011.0
  2010-01-01: 5963.0
  2009-01-01: 5875.0
  2008-01-01: 5852.0
  2007-01-01: 5687.0
  2006-01-01: 5614.0
  2005-01-01: 4412.0
  2004-01-01: 4334.0
  2003-01-01: 4262.0
  2002-01-01: 4191.0
  2001-01-01: 4121.0
  2000-01-01: 26600.0
  1999-01-01: 26230.0
  1998-01-01: 26100.0
  1997-01-01: 25890.0
  1996-01-01: 25610.0
  1995-01-01: 22610.0
  1994-01-01: 22210.0
  1993-01-01: 21900.0
  1992-01-01: 19220.0
  1991-01-01: 18690.0
  1990-01-01: 18140.0
  1989-01-01: 17570.0
  1988-01-01: 17000.0
  1987-01-01: 16560.0
  1986-01-01: 16030.0
  1985-01-01: 15650.0
  1984-01-01: 14820.0
  1983-01-01: 13770.0
  1982-01-01: 12620.0
  1981-01-01: 11230.0
  1980-01-01: 9890.0
  1979-01-01: 8725.0
  1978-01-01: 7925.0
  1977-01-01: 7250.0
  1976-01-01: 6725.0
  1975-01-01: 6125.0
  1974-01-01: 5500.0
  1973-01-01: 4950.0
  1972-01-01: 3300.0
  1971-01-01: 3100.0
  1970-01-01: 2900.0
  1969-01-01: 2700.0
  1965-01-01: 2500.0
  1

## Création d'un scénario

In [116]:
scenario.init_single_entity(
    period = '2017',
    parent1 = dict(
        age = 40,
        salaire_de_base = 50000,
        categorie_salarie = 'prive_cadre' 
        ),
    enfants = [
        dict(age = 1),
        dict(age = 10),
        ],
    )

In [117]:
for entity in tax_benefit_system.entities:
    print entity.key
    for name, variable in tax_benefit_system.variables.iteritems(): 
        if variable.entity == entity:
            if not variable.formula.dated_formulas_class:
                print '  - ', name 

individu
  -  ressortissant_eee
  -  forfait_jours_remuneres_volume
  -  prise_en_charge_employeur_retraite_supplementaire
  -  indemnites_journalieres_adoption
  -  aacc_pvce
  -  mncn_impo
  -  nbnc_defi
  -  complementaire_sante_taux_employeur
  -  mbic_pvct
  -  valeur_locative_terrains_non_loue
  -  mbic_pvce
  -  cotisation_sociale_mode_recouvrement
  -  prestation_compensatoire
  -  contrat_de_travail_debut
  -  dependance_tarif_etablissement_gir_5_6
  -  primes_salaires
  -  scolarite
  -  f5sq
  -  garde_alternee
  -  arag_impg
  -  mbnc_exon
  -  nrag_defi
  -  bourse_enseignement_sup
  -  echelon_bourse
  -  pensions_alimentaires_versees_individu
  -  mncn_mvlt
  -  mva
  -  revenus_locatifs
  -  prevoyance_obligatoire_cadre_taux_employe
  -  salaire_de_base
  -  aeeh_niveau_handicap
  -  cbnc_assc
  -  mbnc_mvct
  -  indemnites_forfaitaires
  -  dependance_plan_aide_domicile
  -  interets_epargne_sur_livrets
  -  enceinte
  -  chomeur_longue_duree
  -  nacc_meup
  -  nacc_e

## Création d'une simulation

In [118]:
simulation = scenario.new_simulation()

### Calcul de l'impot sur le revenu (IRPP)

In [119]:
simulation.calculate('irpp', '2017')

array([-3935.19677734], dtype=float32)

In [120]:
simulation.calculate('irpp', '2017')[0].round()

-3935.0

In [121]:
simulation.calculate('revenu_disponible', '2017')[0].round()

49454.0

In [122]:
simulation.calculate('salaire_de_base', '2017-01')[0].round()

4167.0

### Calculer le revenu net


In [56]:
simulation.calculate('revenu_net', '2017')[0].round()

46651.0

In [54]:
simulation.calculate('cotisations_employeur', '2017-01')

array([-1757.58850098,     0.        ,     0.        ], dtype=float32)

In [39]:
simulation.calculate('af', '2017-01')

array([ 130.11999512], dtype=float32)

In [41]:
simulation.calculate('ppa', '2017-01')

array([ 0.], dtype=float32)

## Test d'une réforme

In [147]:
from openfisca_core import reforms

In [148]:
tax_benefit_system.parameters.impot_revenu.bareme.brackets[1].rate

2014-01-01: 0.14
2006-01-01: 0.055
2003-01-01: 0.0683
2002-01-01: 0.0705
2001-01-01: 0.075
2000-01-01: 0.0825
1999-01-01: 0.095
1996-01-01: 0.105
1993-01-01: 0.12
1973-01-01: 0.05
1948-01-01: 0.1
1945-01-01: 0.12

In [151]:
def modify_parameters(parameters):
    reform_period = periods.period("2017-01-01")
    parameters.impot_revenu.bareme.brackets[1].rate.update(period = reform_period, value = 0.30)
    return parameters

class modif_irpp(Reform):
    name = u'Increase rate of irpp'

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

NameError: name 'Reform' is not defined